In [ ]:
#importaciones
from IPython.display import HTML
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import _converter
%matplotlib inline 
plt.style.use('default')
import datetime
import math
sns.set(style="whitegrid") # seteando tipo de grid en seaborn

In [ ]:
df_postulaciones = pd.read_csv("../datos_navent_fiuba/fiuba_4_postulaciones.csv", parse_dates=['fechapostulacion'])
df_avisos_detalle = pd.read_csv("../datos_navent_fiuba/fiuba_6_avisos_detalle.csv")

In [ ]:
df_posts_avisos = df_avisos_detalle.merge(df_postulaciones, on='idaviso')
df_posts_avisos['fecha'] = df_posts_avisos['fechapostulacion'].dt.date

In [ ]:
def graficar_top_n_postulaciones(N, clave_agrupacion, clave_titulo, x_tick_step=1):
    df_posts_agrupado =  df_posts_avisos.groupby(clave_agrupacion)\
                                        .agg({'idpostulante':'count'})\
                                        .rename(columns={'idpostulante': 'count'})

    tot = df_posts_agrupado['count'].sum()

    df_posts_agrupado['perc'] = df_posts_agrupado['count'] * 100.0 / tot
    df_posts_agrupado = df_posts_agrupado.sort_values(by='count', ascending=False)
    df_posts_agrupado = df_posts_agrupado.reset_index()

    df_top_posts = pd.DataFrame(df_posts_agrupado.head(N))

    max_perc = math.ceil(df_top_posts['perc'].max())
    xticks = np.arange(0, max_perc, x_tick_step)

    g = sns.barplot(x=df_top_posts['perc'], y=df_top_posts[clave_agrupacion], orient='h',
                    palette=sns.color_palette("hls", N))
    g.set_title("Top {0} de postulaciones por {1}".format(N, clave_titulo.lower()), fontsize=15)
    g.set_xlabel("Postulaciones (%)", fontsize=12)
    g.set_ylabel(clave_titulo, fontsize=12)
    g.set(xticks=xticks);
    
    return g

In [ ]:
graficar_top_n_postulaciones(10, 'nombre_area', 'Area')

In [ ]:
graficar_top_n_postulaciones(20, 'denominacion_empresa', 'Empresa', x_tick_step=0.5)

In [ ]:
# TODO: ver si se puede hacer alguna grafica con la evolucion en el tiempo
df = df_posts_avisos.groupby(['nombre_area', 'fecha'])\
                    .size()\
                    .reset_index()\
                    .rename(columns={0:'count'})
tot = df['count'].sum()
df['perc'] = df['count'] / tot

pivoted = pd.pivot_table(df, values='perc', columns='nombre_area', \
                         index='fecha')
pivoted = pivoted.divide(pivoted.sum(axis=1), axis=0)

f = plt.figure()
plt.title('Postulaciones por area a lo largo del tiempo (%)', color='black')
ax = pivoted.plot(kind='area', stacked=True,figsize=(16,8),ax=f.gca())
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles[::-1], labels[::-1],loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set_ylabel('Porcentaje (%)');
ax.set_xlabel('Fecha');
ax.set_ylim(0,1)
plt.show()